In [ ]:
import string
from pathlib import Path

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import yaml
from utils import (
    add_to_variables,
    format_dictionary_for_yaml,
    load_data,
    plot_zero_zero,
)

In [ ]:
plt.style.use("default")
plot_kwargs = {"edgecolors": "black", "linewidths": 0.5}
mpl.rcParams["font.size"] = 14

Load DataFrames

In [ ]:
path_interpolation = snakemake.input.data_interpolation
path_no_interpolation = snakemake.input.data_no_interpolation
path_azaphenalenes_ref = snakemake.input.reference

In [ ]:
df_interpolation = load_data(path_interpolation, path_azaphenalenes_ref).dropna()
df_no_interpolation = load_data(path_no_interpolation, path_azaphenalenes_ref).dropna()

In [ ]:
df_merged = pd.merge(
    df_interpolation,
    df_no_interpolation,
    left_index=True,
    right_index=True,
    suffixes=("_interpolation", "_no_interpolation"),
)

Plot results

In [ ]:
variables = {}
label_calculation = "interpolation"

jobs = (
    ("SCF + DSP", "scf", "t1_s1_dsp_scf"),
    ("CIS + DSP", "cis", "t1_s1_dsp_cis"),
)

fig, ax = plt.subplots(1, 2, figsize=plt.rcParams["figure.figsize"] * np.array([2, 1]))

# Create subfigure labels
for i, ax_ in enumerate(ax):
    label = string.ascii_lowercase[i]
    ax_.text(
        0.05,
        0.95,
        f"({label})",
        fontsize="large",
        transform=ax_.transAxes,
        verticalalignment="top",
    )

for i, (label_plot, label_method, name) in enumerate(jobs):
    print(name)
    x_name = name + "_no_interpolation"
    y_name = name + "_interpolation"

    _, _, results = plot_zero_zero(
        df_merged,
        x_name=x_name,
        y_name=y_name,
        plot_type="scatter",
        ax=ax[i],
        legend_loc="lower right",
        zero_zero=False,
        n_dec=3,
    )
    ax[i].set_label(label_plot)
    variables = add_to_variables(
        variables,
        results,
        label_calculation=label_calculation,
        label_compound="all",
        label_method=label_method,
    )
display(fig)
fig.tight_layout()

path_figure = snakemake.output.figure_interpolation
variables[f"fig_interpolation"] = "../" + str(Path(path_figure).with_suffix(""))
fig.savefig(path_figure)

Save variables for manuscript

In [ ]:
path_variables = snakemake.output.variables
with open(path_variables, "w") as f:
    yaml.dump(format_dictionary_for_yaml(variables, n_dec=3), f)